<a href="https://colab.research.google.com/github/AabidMK/RealtyAI_Infosys_Internship_Aug2025/blob/Arnab-Jana/Notebook_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Path setup
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks"
RAW_CSV = f"{DATA_DIR}/Real Estate Data V21.csv"   # update if filename differs
CLEAN_CSV = f"{DATA_DIR}/clean_real_estate.csv"

import os
print("Files in", DATA_DIR, ":\n", os.listdir(DATA_DIR))


Mounted at /content/drive
Files in /content/drive/MyDrive/Colab Notebooks :
 ['Real Estate Data V21.csv', 'Untitled0.ipynb', 'Untitled1.ipynb', 'Untitled2.ipynb', 'Untitled', 'RealtyAI_Smart_RealEstate_Insight_Platform.ipynb', 'Untitled3.ipynb', 'Untitled4.ipynb']


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Path setup
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks"
RAW_CSV = f"{DATA_DIR}/Real Estate Data V21.csv"   # change name if different
CLEAN_CSV = f"{DATA_DIR}/clean_real_estate.csv"

import os
print("Files in", DATA_DIR, ":\n", os.listdir(DATA_DIR))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in /content/drive/MyDrive/Colab Notebooks :
 ['Real Estate Data V21.csv', 'Untitled0.ipynb', 'Untitled1.ipynb', 'Untitled2.ipynb', 'Untitled', 'RealtyAI_Smart_RealEstate_Insight_Platform.ipynb', 'Untitled3.ipynb', 'Untitled4.ipynb']


In [4]:
import pandas as pd, numpy as np

df = pd.read_csv(RAW_CSV)
print("Original shape:", df.shape)
df.head()


Original shape: (14528, 9)


,Name,Property Title,Price,Location,Total_Area,Price_per_SQFT,Description,Baths,Balcony
0,Casagrand ECR 14,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",₹1.99 Cr,"Kanathur Reddikuppam, Chennai",2583,7700.0,Best 4 BHK Apartment for modern-day lifestyle ...,4,Yes
1,"Ramanathan Nagar, Pozhichalur,Chennai",10 BHK Independent House for sale in Pozhichal...,₹2.25 Cr,"Ramanathan Nagar, Pozhichalur,Chennai",7000,3210.0,Looking for a 10 BHK Independent House for sal...,6,Yes
2,DAC Prapthi,"3 BHK Flat for sale in West Tambaram, Chennai",₹1.0 Cr,"Kasthuribai Nagar, West Tambaram,Chennai",1320,7580.0,"Property for sale in Tambaram, Chennai. This 3...",3,No
3,"Naveenilaya,Chepauk, Triplicane,Chennai",7 BHK Independent House for sale in Triplicane...,₹3.33 Cr,"Naveenilaya,Chepauk, Triplicane,Chennai",4250,7840.0,Entire Building for sale with 7 units of singl...,5,Yes
4,VGN Spring Field Phase 1,"2 BHK Flat for sale in Avadi, Chennai",₹48.0 L,"Avadi, Chennai",960,5000.0,"Property for sale in Avadi, Chennai. This 2 BH...",3,Yes


In [5]:
import re

def convert_price(p):
    if pd.isna(p):
        return np.nan
    s = str(p).replace("₹","").replace(",","").strip().lower()
    bad_tokens = ["acs","onwards","negotiable","ask","call","contact","price on"]
    if any(tok in s for tok in bad_tokens):
        return np.nan
    nums = re.findall(r"[\d\.]+", s)
    if not nums:
        return np.nan
    try:
        if "cr" in s:
            return float(nums[0]) * 100.0
        if "l" in s and "sq" not in s:
            return float(nums[0])
        return float(nums[0])
    except:
        return np.nan

df["Price_num"] = df["Price"].apply(convert_price)
print("Valid prices:", df["Price_num"].notna().sum(), "/", len(df))
df[["Price","Price_num"]].head(20)


Valid prices: 14527 / 14528


,Price,Price_num
0,₹1.99 Cr,199.00
1,₹2.25 Cr,225.00
2,₹1.0 Cr,100.00
3,₹3.33 Cr,333.00
4,₹48.0 L,48.00
5,₹40.0 L,40.00
6,₹60.0 L,60.00
7,₹72.35 L,72.35
8,₹42.0 L,42.00
9,₹30.0 L,30.00


In [6]:
# Balcony -> binary
df["Balcony"] = df["Balcony"].astype(str).str.strip().str.lower()
df["Balcony"] = df["Balcony"].replace({"yes":1,"no":0,"y":1,"n":0,"1":1,"0":0})
df.loc[~df["Balcony"].isin([0,1]), "Balcony"] = np.nan

# Baths
df["Baths"] = pd.to_numeric(df["Baths"], errors="coerce")

# Total_Area + Price_per_SQFT
df["Total_Area"] = pd.to_numeric(df["Total_Area"], errors="coerce")
df["Price_per_SQFT"] = pd.to_numeric(df["Price_per_SQFT"], errors="coerce")

mask = df["Price_per_SQFT"].isna() & df["Price_num"].notna() & df["Total_Area"].notna() & (df["Total_Area"]>0)
df.loc[mask, "Price_per_SQFT"] = (df.loc[mask,"Price_num"]*100000.0)/df.loc[mask,"Total_Area"]

# BHK extraction
df["BHK"] = df["Property Title"].astype(str).str.extract(r'(\d+)\s*BHK', expand=False)
df["BHK"] = pd.to_numeric(df["BHK"], errors="coerce")

# City extraction
df["City"] = df["Location"].astype(str).str.split(",").str[-1].str.strip()


/tmp/ipython-input-2833295624.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Balcony"] = df["Balcony"].replace({"yes":1,"no":0,"y":1,"n":0,"1":1,"0":0})


In [7]:
# Drop impossible values
df = df[df["Price_num"].notna()]
df = df[df["Total_Area"].notna()]
df = df[(df["Total_Area"]>50) & (df["Total_Area"]<50000)]
df = df[df["Price_num"]>1]
df = df[(df["Baths"].isna()) | ((df["Baths"]>=1)&(df["Baths"]<=20))]

print("Shape after filtering:", df.shape)

# Outlier clipping
def iqr_clip(series, k=1.5):
    q1, q3 = series.quantile(0.25), series.quantile(0.75)
    iqr = q3-q1
    low, high = q1-k*iqr, q3+k*iqr
    return series.clip(low, high)

df["Price_num"] = iqr_clip(df["Price_num"])
df["Total_Area"] = iqr_clip(df["Total_Area"])


Shape after filtering: (14520, 12)


In [8]:
for col in ["Baths","Balcony","Price_per_SQFT","BHK"]:
    df[col] = df[col].fillna(df[col].median())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Location_enc"] = le.fit_transform(df["Location"].astype(str))

print("Unique locations:", len(le.classes_))
df.head()


Unique locations: 7048


,Name,Property Title,Price,Location,Total_Area,Price_per_SQFT,Description,Baths,Balcony,Price_num,BHK,City,Location_enc
0,Casagrand ECR 14,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",₹1.99 Cr,"Kanathur Reddikuppam, Chennai",2583.0,7700.0,Best 4 BHK Apartment for modern-day lifestyle ...,4,1.0,199.0,4.0,Chennai,2733
1,"Ramanathan Nagar, Pozhichalur,Chennai",10 BHK Independent House for sale in Pozhichal...,₹2.25 Cr,"Ramanathan Nagar, Pozhichalur,Chennai",2622.5,3210.0,Looking for a 10 BHK Independent House for sal...,6,1.0,225.0,10.0,Chennai,4729
2,DAC Prapthi,"3 BHK Flat for sale in West Tambaram, Chennai",₹1.0 Cr,"Kasthuribai Nagar, West Tambaram,Chennai",1320.0,7580.0,"Property for sale in Tambaram, Chennai. This 3...",3,0.0,100.0,3.0,Chennai,2797
3,"Naveenilaya,Chepauk, Triplicane,Chennai",7 BHK Independent House for sale in Triplicane...,₹3.33 Cr,"Naveenilaya,Chepauk, Triplicane,Chennai",2622.5,7840.0,Entire Building for sale with 7 units of singl...,5,1.0,229.5,7.0,Chennai,3895
4,VGN Spring Field Phase 1,"2 BHK Flat for sale in Avadi, Chennai",₹48.0 L,"Avadi, Chennai",960.0,5000.0,"Property for sale in Avadi, Chennai. This 2 BH...",3,1.0,48.0,2.0,Chennai,627


In [9]:
df.to_csv(CLEAN_CSV, index=False)
print("✅ Cleaned data saved to:", CLEAN_CSV)


✅ Cleaned data saved to: /content/drive/MyDrive/Colab Notebooks/clean_real_estate.csv
